### 공동주택 코드 매핑

In [570]:
# 라이브러리
import os
import chardet
import pandas as pd
import re 

p = re.compile('\(([^)]+)')  

def jibun_juso(addr):
    d = p.findall(addr) 
    if isinstance(d, list) and len(d) > 0:
        return d[0]
    return None

def doro_juso(addr):
    p = addr.find("(")
    juso = addr[0:p-1].rstrip()
    return juso

def part_back(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        part_front = obj.split()[0]
        part_back = obj.replace(part_front, "").lstrip()
        return part_back
    else:
        return None
    
def part_front(obj):
    part_front = obj[1].replace(obj[0], "").rstrip()
    return part_front

def no_blank(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        re_str = obj.replace(" ", "")
        return re_str
    else:
        return None

In [571]:
# 데이터 파일 위치
data_root = "D:\PythonProject\data-gatherer\molit_apt\data"
# 아파트 코드 목록: 외부(국토교통부)
file_ext = os.path.join(data_root, "공동주택_국토부_apt_info_20240411.csv")
# 아파트 코드 목록: 내부(고객지원시스템)
file_int = os.path.join(data_root, "공동주택_고객지원시스템_20240415.csv")

# 파일의 인코딩 확인
with open(file_ext, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_ext = result['encoding']
    
with open(file_int, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_int = result['encoding']

* df_int

In [572]:
# 파일 데이터 읽기
data_int = pd.read_csv(file_int, encoding=encoding_int, dtype=object)
df_int = data_int.copy()
int_cnt = df_int.shape[0]

In [573]:
""" 데이터 정리 """
# 1. 공동주택이 아닌 데이터 제외
idx = df_int[df_int['용도'] != '공동주택'].index
df_int.drop(idx, inplace=True)
# 2. 필요한 항목만 남김
df_int = df_int[['아파트명', '아파트코드', '우편번호', '주소']]
# 3. 중복 데이터 없애기
df_int.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)
# 4. 시군구 항목 생성
df_int['시군구'] = df_int['주소'].apply(lambda x: x.split()[0])
# 5. 대구광역시 공급권역 데이터만 남기고 나머지(경북 지역) 데이터 삭제
idx = df_int[~df_int['시군구'].isin(['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군'])].index
df_int.drop(idx, inplace=True)
# 6. 지번주소 항목 분리 생성
df_int['지번주소'] = df_int['주소'].apply(jibun_juso)
df_int['지번주소_공백X'] = df_int['지번주소'].apply(no_blank)
# 7. 도로명주소 항목 분리 생성
df_int['시군구_도로명주소'] = df_int['주소'].apply(doro_juso)
df_int['도로명주소'] = df_int['시군구_도로명주소'].apply(part_back)
df_int['도로명주소_공백X'] = df_int['도로명주소'].apply(no_blank)
# 8. 아파트명 공백X
df_int['아파트명_공백X'] = df_int['아파트명'].apply(lambda x: x.replace(" ",""))
# 9. 필요항 컬럼만 구성
df_int = df_int[['아파트코드','아파트명', '아파트명_공백X', '시군구', '지번주소', '지번주소_공백X', '도로명주소', '도로명주소_공백X']]

In [574]:
df_int.head(3)

,아파트코드,아파트명,아파트명_공백X,시군구,지번주소,지번주소_공백X,도로명주소,도로명주소_공백X
0,201425,청라힐지웰더센트로,청라힐지웰더센트로,중구,대신동 103-9,대신동103-9,달성로 71,달성로71
1,201417,힐스테이트앞산센트럴,힐스테이트앞산센트럴,남구,봉덕동 1067-35,봉덕동1067-35,대덕로34길 10-10,대덕로34길10-10
2,201382,오페라센텀파크서한이다음,오페라센텀파크서한이다음,북구,고성동3가 6-20,고성동3가6-20,고성로37길 1,고성로37길1


* df_ext

In [575]:
# 파일 데이터 읽기
data_ext = pd.read_csv(file_ext, encoding=encoding_ext, dtype=object)
df_ext = data_ext.copy()
ext_cnt = df_ext.shape[0]

In [576]:
# df_ext.isnull().sum()

In [577]:
""" 데이터 정리 """
# 1. 필요한 항목만 남김
df_ext = df_ext[['kapt_cd', 'kapt_nm', 'as2', 'as3', 'as4', 'kapt_addr', 'doro_juso']]
# 2. 시군구_도로명주소 항목 생성
df_ext['시군구_도로명주소'] = df_ext['doro_juso'].str.lstrip('대구광역시')
# 3. 도로명주소 항목 생성
df_ext['도로명주소'] = df_ext['시군구_도로명주소'].apply(part_back)
df_ext['도로명주소_공백X'] = df_ext['도로명주소'].apply(no_blank)
# 4. 법정동주소 항목 생성   
df_ext['광역시_법정동주소'] = df_ext[['kapt_nm', 'kapt_addr']].apply(part_front, axis=1)
df_ext['시군구_법정동주소'] = df_ext['광역시_법정동주소'].apply(part_back)
df_ext['법정동주소'] = df_ext['시군구_법정동주소'].apply(part_back)
df_ext['법정동주소_공백X'] = df_ext['법정동주소'].apply(no_blank)
# 5. 컬럼명 변경
df_ext.rename(columns={'kapt_nm' : '아파트명', 'as2' : '시군구'}, inplace=True)
# 6. 아파트명 공백X
df_ext['아파트명_공백X'] = df_ext['아파트명'].apply(lambda x: x.replace(" ",""))
# 7. 필요한 컬럼만 구성
df_ext = df_ext[['kapt_cd', '아파트명', '아파트명_공백X', '시군구', 'as3', 'as4', '도로명주소', '도로명주소_공백X', '법정동주소', '법정동주소_공백X']]

In [578]:
df_ext

,kapt_cd,아파트명,아파트명_공백X,시군구,as3,as4,도로명주소,도로명주소_공백X,법정동주소,법정동주소_공백X
0,A10022694,북구청역푸르지오에듀포레,북구청역푸르지오에듀포레,북구,노원동1가,NaN,침산남로 30,침산남로30,노원동1가 0,노원동1가0
1,A10022700,동대구역 센텀 화성파크드림,동대구역센텀화성파크드림,동구,신암동,NaN,아양로 133,아양로133,신암동 0,신암동0
2,A10022702,힐스테이트대구역오페라,힐스테이트대구역오페라,북구,고성동1가,NaN,칠성남로 30,칠성남로30,고성동1가 0,고성동1가0
3,A10022714,수성푸르지오리버센트 아파트,수성푸르지오리버센트아파트,수성구,중동,NaN,수성로35길 50,수성로35길50,중동 556,중동556
4,A10022719,수성해모로하이엔,수성해모로하이엔,수성구,파동,NaN,파동로 47,파동로47,파동 540-14,파동540-14
...,...,...,...,...,...,...,...,...,...,...
1034,A71185711,논공성원아파트,논공성원아파트,달성군,논공읍,북리,논공로23길 14,논공로23길14,논공읍 북리 803-4,논공읍북리803-4
1035,A71186002,달성용계주공,달성용계주공,달성군,가창면,용계리,가창로216길 25,가창로216길25,가창면 용계리 34-2,가창면용계리34-2
1036,A71186503,가창중석타운,가창중석타운,달성군,가창면,용계리,가창로 1008,가창로1008,가창면 용계리 465,가창면용계리465
1037,A71187301,학산아파트,학산아파트,달성군,현풍읍,중리,현풍동로 23,현풍동로23,현풍읍 중리 335,현풍읍중리335


In [579]:
df_ext.head(3)

,kapt_cd,아파트명,아파트명_공백X,시군구,as3,as4,도로명주소,도로명주소_공백X,법정동주소,법정동주소_공백X
0,A10022694,북구청역푸르지오에듀포레,북구청역푸르지오에듀포레,북구,노원동1가,NaN,침산남로 30,침산남로30,노원동1가 0,노원동1가0
1,A10022700,동대구역 센텀 화성파크드림,동대구역센텀화성파크드림,동구,신암동,NaN,아양로 133,아양로133,신암동 0,신암동0
2,A10022702,힐스테이트대구역오페라,힐스테이트대구역오페라,북구,고성동1가,NaN,칠성남로 30,칠성남로30,고성동1가 0,고성동1가0


In [580]:
print(df_ext.shape, df_int.shape)

(1039, 10) (2391, 8)


* 1차 매칭 - 도로명주소

In [581]:
# 1차 매칭
df_merge1 = pd.merge(df_ext, df_int, how='outer', left_on="도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match1 = df_merge1[df_merge1['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match1['match'] = '1. 도로명주소 일치'
# 1차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_merge1[df_merge1['_merge'] == 'both']['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_merge1[df_merge1['_merge'] == 'both']['아파트코드'])].index
df_int.drop(idx_int, inplace=True)

* 2차 매칭 - 아파트명

In [582]:
# 2차 매칭
df_merge2 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X", right_on="아파트명_공백X", suffixes=('_left','_right'), indicator=True)
df_match2 = df_merge2[df_merge2['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match2['match'] = '2. 아파트명 일치'
# 2차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_merge2[df_merge2['_merge'] == 'both']['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_merge2[df_merge2['_merge'] == 'both']['아파트코드'])].index
df_int.drop(idx_int, inplace=True)

* 3차 매칭 - [EXT]법정동주소 == [INT]지번주소

In [583]:
# 3차 매칭
df_merge3 = pd.merge(df_ext, df_int, how='outer', left_on="법정동주소_공백X", right_on="지번주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match3 = df_merge3[df_merge3['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match3['match'] = '3. 국토부_법정동주소와 내부시스템_지번주소 일치'
# 3차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_merge3[df_merge3['_merge'] == 'both']['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_merge3[df_merge3['_merge'] == 'both']['아파트코드'])].index
df_int.drop(idx_int, inplace=True)

* 4차 매칭

In [584]:
# 4차 매칭
df_ext['as3_도로명주소_공백X'] = df_ext['as3'] + df_ext['도로명주소_공백X']
df_merge4 = pd.merge(df_ext, df_int, how='outer', left_on="as3_도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match4 = df_merge4[df_merge4['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match4['match'] = '4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치'
# 4차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_merge4[df_merge4['_merge'] == 'both']['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_merge4[df_merge4['_merge'] == 'both']['아파트코드'])].index
df_int.drop(idx_int, inplace=True)

* 총합

In [585]:
m1 = df_match1.shape[0]
m2 = df_match2.shape[0]
m3 = df_match3.shape[0]
m4 = df_match4.shape[0]
total = m1 + m2 + m3 + m4
print(f"대구광역시 국토부 데이터:\t {ext_cnt}")
print(f"고객지원시스템 데이터:\t {int_cnt}")
print(f"매칭: 1차 매칭 {m1}, 2차 매칭 {m2}, 3차 매칭 {m3}, 4차 매칭 {m4}, 총 매칭 {total}, 잔여 건수(국토부 데이터 기준) {total - ext_cnt}")

대구광역시 국토부 데이터:	 1039
고객지원시스템 데이터:	 3170
매칭: 1차 매칭 875, 2차 매칭 110, 3차 매칭 23, 4차 매칭 43, 총 매칭 1051, 잔여 건수(국토부 데이터 기준) 12


In [586]:
# 매칭 데이터 병합
df_list = [df_match1, df_match2, df_match3, df_match4]
df_all = pd.concat(df_list, ignore_index=True)

In [587]:
df_all.sort_values(by='match')['match'].value_counts()

match
1. 도로명주소 일치                        875
2. 아파트명 일치                         110
4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치     43
3. 국토부_법정동주소와 내부시스템_지번주소 일치         23
Name: count, dtype: int64

In [594]:
file_ext_unmatched = os.path.join(data_root, "공동주택_국토부_apt_info_20240411_unmatched.xlsx")
df_ext.to_excel(file_ext_unmatched, index=False)

In [589]:
# 매칭되지 못한 건 검색
df_int[df_int['아파트명'].str.contains('진천역')]

,아파트코드,아파트명,아파트명_공백X,시군구,지번주소,지번주소_공백X,도로명주소,도로명주소_공백X
1786,200066,진천역화성파크리젠시1단지,진천역화성파크리젠시1단지,달서구,유천동 120-7,유천동120-7,월배로11길 9-15,월배로11길9-15


* 원 데이터 검색

In [590]:
data_ext.columns

Index(['sido_cd', 'kapt_cd', 'kapt_nm', 'as1', 'as2', 'as3', 'as4', 'stdg_cd',
       'kapt_addr', 'cd_sale_nm', 'cd_heat_nm', 'kapt_tarea', 'kapt_dong_cnt',
       'kapt_da_cnt', 'kapt_cnst_co', 'kapt_dev_co', 'kapt_tel', 'kapt_fax',
       'kapt_url', 'cd_apt_nm', 'doro_juso', 'ho_cnt', 'cd_mgr_nm',
       'cd_hall_nm', 'kapt_use_dt', 'kapt_marea', 'kapt_mparea_60',
       'kapt_mparea_85', 'kapt_mparea_135', 'kapt_mparea_136', 'priv_area',
       'cr_dt', 'cr_id'],
      dtype='object')

In [596]:
data_ext[data_ext['kapt_nm'].str.contains('서대구역')][['kapt_nm', 'kapt_addr','doro_juso']]

,kapt_nm,kapt_addr,doro_juso
12,서대구역화성파크드림,대구광역시 서구 평리동 1512-10 서대구역화성파크드림,대구광역시 서구 서대구로29길 30
20,서대구역반도유보라센텀,대구광역시 서구 평리동 1083-2 서대구역반도유보라센텀,대구광역시 서구 문화로 230
46,서대구역서한이다음 더 퍼스트,대구광역시 서구 평리동 1481-10 서대구역서한이다음 더 퍼스트,NaN


In [597]:
data_int[data_int['아파트명'].str.contains('서대구역')]

,아파트명,아파트코드,세대수,우편번호,주소,난방구분,용도,공동주택계약,설치위치,서비스센터,Unnamed: 10
7,서대구역서한이다음더퍼스트1단지,201342,566,41765,서구 국채보상로37길 38 (평리동 1497-44),개별난방,공동주택,아니오,실내기타,서비스2센터(주),NaN
386,서대구역서한이다음더퍼스트2단지,201364,290,41765,서구 국채보상로37길 38 (평리동 1497-44),개별난방,공동주택,아니오,실내기타,서비스2센터(주),NaN
388,서대구역화성파크드림3단지,201357,184,41766,서구 서대구로29길 30 (평리동 1515-1),개별난방,공동주택,아니오,실내기타,서비스2센터(주),NaN
1531,서대구역화성파크드림2단지,201356,497,41767,서구 서대구로29길 30 (평리동 1511-4),개별난방,공동주택,아니오,실내기타,서비스2센터(주),NaN
1532,서대구역화성파크드림1단지,201355,913,41768,서구 서대구로29길 30 (평리동 1505-4),개별난방,공동주택,아니오,실내기타,서비스2센터(주),NaN
1949,서대구역반도유보라센텀,201323,"1,678",41776,서구 문화로 230 (평리동 1083-2),개별난방,공동주택,아니오,베란다,서비스2센터(주),NaN
